# Airbnb Capstone Project

## Import Libraries

In [1]:
### import all libraries and set settings 
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

import requests

from py_functions import increase_bbox
from py_functions import get_dataframe
from py_functions import get_engine
from sklearn.neighbors import BallTree
from scipy.spatial import cKDTree
from shapely.geometry import Point
from shapely.ops import transform
from functools import partial

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from bs4 import BeautifulSoup

import threading
from threading import Thread

from concurrent.futures import ThreadPoolExecutor, process, wait

pd.set_option('display.max_columns', None) # show all columns  

## Download osm_short from postgresql database

In [2]:
engine = get_engine()
schema = 'hh_analytics_23_1' # your course schema name, example 'hh_analytics_22_1
table_name = 'g1_osm_short'

In [3]:
osm_short = get_dataframe(f'SELECT * FROM {schema}.{table_name}')

In [4]:
osm_short.head(2)

,id,latitude,longitude,name,amenity,cuisine,diet_vegetarian,diet_vegan,railway,address,gastronomy
0,451152,51.600840,-0.194608,King of Prussia,bar,pizza;burger,True,True,None,363 Regents Park Road London N3 1DH,False
1,451153,51.602031,-0.193503,Central Restaurant,restaurant,None,None,None,None,None,True


## Restaurant scraping

In [5]:
# New DF for bars with addresses and names to fill with reviews
bar_address = osm_short[(osm_short["amenity"] == "bar") & (~osm_short["address"].isna()) & (~osm_short["name"].isna())]
bar_address = bar_address.reset_index().drop(columns="index")

In [6]:
# Uncomment both only the first time this is run.
# bar_reviews = pd.DataFrame(columns=["id","name","rating","reviews","price","closed","url"])
# bar_reviews.to_csv("data/london/bar_address.csv")

In [36]:
# THIS ONE, "bar_address" must be csv on computer
# bars, address not nan, web scraping loop with threading

bar_reviews = pd.read_csv("data/london/bar_reviews.csv", index_col="Unnamed: 0")
bar_address_remaining = bar_address[~bar_address["id"].isin(bar_reviews["id"].to_list())]


service = Service(ChromeDriverManager().install())

def driver_setup():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def get_rating(i):
    global bar_reviews
    bar_id = bar_address_remaining["id"].iloc[i]
    bar_name = bar_address_remaining["name"].iloc[i]
    driver=driver_setup()
    url = f'http://maps.google.com/?q={bar_address_remaining["name"].iloc[i]}+{bar_address_remaining["address"].iloc[i]}'
    driver.get(url)
    time.sleep(1)


    # Press accept all on cookie question
    try:
        driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()
        time.sleep(1)
    except NoSuchElementException:
        pass
    
    # Extract raiting/num of reviews, price level and if permanently closed
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        case_reviews = soup.find('div', class_ = 'F7nice')
        case_rating = case_reviews.text.split("(")[0]
        case_reviews = case_reviews.text.split("(")[1][:-1]
        print(i, bar_address_remaining["name"].iloc[i], case_rating)
        print(i, bar_address_remaining["name"].iloc[i], case_reviews)
    except Exception:
        case_rating = np.nan
        case_reviews = np.nan
        print(i, bar_address_remaining["name"].iloc[i], "Rating no")
        pass
   
    try:
        case_closed = str("Temporarily closed" in soup.find('div', class_ = 'skqShb').text)
        print(i, bar_address_remaining["name"].iloc[i], case_closed)
    except Exception:
        case_closed = np.nan
        print(i, bar_address_remaining["name"].iloc[i], "Closed no")
        pass

    try:
        case_price = soup.find('span', class_ = 'mgr77e').text[1:]
        print(i, bar_address_remaining["name"].iloc[i], case_price)
    except Exception:
        case_price = np.nan
        print(i, bar_address_remaining["name"].iloc[i], "Euro no")
        pass
   
        

    poi_url = url
    row = {"id": bar_id, "name": bar_name, "rating": case_rating, "reviews": case_reviews, "price": case_price, "closed": case_closed, "url": poi_url}
    print(row)
    bar_reviews = pd.concat([bar_reviews, pd.DataFrame(row, index=[i])], ignore_index=True)

    driver.close()
    return bar_reviews

# Insert number of iterations in range
threadlist = []
with ThreadPoolExecutor(max_workers=4) as executor:
    for i in range(1):
        threadlist.append(executor.submit(get_rating, i))
        time.sleep(1)

wait(threadlist)

bar_reviews = bar_reviews.drop_duplicates(subset="id", keep=False)
bar_reviews.to_csv('data/london/bar_reviews.csv')

0 The Rose 4.2
0 The Rose 1,389
0 The Rose False
0 The Rose €€
{'id': 1168160316, 'name': 'The Rose', 'rating': '4.2', 'reviews': '1,389', 'price': '€€', 'closed': 'False', 'url': 'http://maps.google.com/?q=The Rose+35 Albert Embankment  SE1 7TL'}


In [37]:
bar_reviews.tail()

,id,name,rating,reviews,price,closed,url
3508,316763495,Dirty Martini St. Pauls,3.7,"1,091",€€,False,http://maps.google.com/?q=Dirty Martini St. Pa...
3509,10862052885,The Black Horse,4.6,37,NaN,False,http://maps.google.com/?q=The Black Horse+1 Wi...
3510,146986838,The Long Lane,5.0,77,NaN,False,http://maps.google.com/?q=The Long Lane+52 Lon...
3511,439335533,Lamb and Trotter,4.4,187,NaN,False,http://maps.google.com/?q=Lamb and Trotter+6 L...
3512,1168160316,The Rose,4.2,"1,389",€€,False,http://maps.google.com/?q=The Rose+35 Albert E...


### Upload review data for bars that had addresses

In [ ]:
# closed is only NaN if the place was not found in the google search, otherwise it would be True or False
bar_reviews = bar_reviews[(~bar_reviews["closed"].isna())]

,id,name,rating,reviews,price,closed,url
0,451152,King of Prussia,4.5,291,NaN,False,http://maps.google.com/?q=King of Prussia+363 ...
1,451154,The Catcher in the Rye,4.2,811,€€,False,http://maps.google.com/?q=The Catcher in the R...
2,451271,The Tally Ho,4.0,"1,330",€,False,http://maps.google.com/?q=The Tally Ho+749 Hig...
3,12243302,The George,4.1,"1,678",€,False,http://maps.google.com/?q=The George+ High Str...
4,15262028,The Monkey Puzzle,4.5,"1,502",€€,False,http://maps.google.com/?q=The Monkey Puzzle+30...
...,...,...,...,...,...,...,...
3502,1155385349,The Old White Lion,4.3,"1,153",€€,False,http://maps.google.com/?q=The Old White Lion+1...
3503,987866346,The Havelock Tavern,4.4,532,€€,False,http://maps.google.com/?q=The Havelock Tavern+...
3504,1164893696,The Triple Crown,3.9,53,€,False,http://maps.google.com/?q=The Triple Crown+15 ...
3505,1162262556,The Volunteer,4.4,198,€€,False,http://maps.google.com/?q=The Volunteer+46 Chu...


In [ ]:
from py_functions import get_engine
import psycopg2

In [ ]:
engine = get_engine()
schema = 'hh_analytics_23_1' # your course schema name, example 'hh_analytics_22_1
table_name = 'g1_bar_reviews'

In [ ]:
# Write records stored in poi_gastro to SQL database
if engine!=None:
    try:
        bar_reviews.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='fail', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

The g1_bar_reviews table was imported successfully.


In [ ]:
# Give access to all cohort members
group = "student_role_hh_analytics_23_1"
engine = get_engine()
SQL = f"GRANT ALL  ON {schema}.{table_name} to {group}"
with engine.begin() as conn:
    conn.execute(SQL)


### Scraping for bars without or without correct address by lat/long

In [ ]:
bar_no_address = osm_short[(osm_short["amenity"] == "bar") & (osm_short["address"].isna()) & (~osm_short["name"].isna())]

In [ ]:
# No working address
bar_remaining = bar_reviews[(bar_reviews["closed"].isna())]

In [ ]:
bar_no_address = pd.concat([bar_no_address, bar_remaining])

In [ ]:
bar_no_address = bar_no_address.reset_index().drop(columns="index")

In [ ]:
# Uncomment both only the first time
# bar_reviews_no_add = bar_reviews[0:0]
# bar_reviews_no_add.to_csv("data/london/bar_reviews_no_add.csv")

In [ ]:
# Choose from list, "bar_reviews_no_address" must be csv on computer
# bars, address not nan, web scraping loop with threading
pd.options.mode.chained_assignment = "warn"

bar_reviews_no_add = pd.read_csv("data/london/bar_reviews_no_add.csv", index_col="Unnamed: 0")
bar_no_add_remaining = bar_no_address[~bar_no_address["id"].isin(bar_reviews_no_add["id"].to_list())]


service = Service(ChromeDriverManager().install())

def driver_setup():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def get_rating(i):
    global bar_reviews_no_add
    bar_id = bar_no_add_remaining["id"].iloc[i]
    bar_name = bar_no_add_remaining["name"].iloc[i]
    driver=driver_setup()
    url = f'http://maps.google.com/?ll={bar_no_add_remaining["latitude"].iloc[i]},{bar_no_add_remaining["longitude"].iloc[i]}'
    driver.get(url)
    time.sleep(1)


    # Press accept all on cookie question
    try:
        driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()
        time.sleep(1)
    except NoSuchElementException:
        pass

    driver.get(url)

    driver.find_element(By.XPATH,'/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]').send_keys(bar_no_add_remaining["name"].iloc[i])
    driver.find_element(By.XPATH,'/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]').send_keys(Keys.RETURN)
    time.sleep(4)



    # Extract raiting/num of reviews, price level and if permanently closed
    try:
        # Case for immediate result without list appearing
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        case_reviews = soup.find('div', class_ = 'F7nice')
        case_rating = case_reviews.text.split("(")[0]
        case_reviews = case_reviews.text.split("(")[1][:-1]
        #print(i, bar_no_add_remaining["name"].iloc[i], case_rating)
        #print(i, bar_no_add_remaining["name"].iloc[i], case_reviews)
    except Exception:
        try:
            try:
                # Count the number of ads and skip them
                ads = soup.find_all('span', {'class': 'jHLihd'})
                #print(ads)
                len_ads = len(ads)
                ads_first = ads[0]
                #print(bar_no_add_remaining["name"].iloc[i], "len ads", len(ads))
                #print(bar_no_add_remaining["name"].iloc[i], "ads 1", ads[0])
                if len(ads) > 0:
                    try:
                        #print(bar_no_add_remaining["name"].iloc[i], "len ads over 0")
                        click_list = driver.find_elements(By.CLASS_NAME, 'hfpxzc')
                        #print(bar_no_add_remaining["name"].iloc[i], click_list[len(ads)])
                        click_list[len(ads)].click()
                        time.sleep(2)
                        soup = BeautifulSoup(driver.page_source, 'html.parser')
                        case_reviews = soup.find('div', class_ = 'F7nice')
                        case_rating = case_reviews.text.split("(")[0]
                        case_reviews = case_reviews.text.split("(")[1][:-1]
                    except Exception:
                        pass
            except Exception:
                # Case if no ads
                #print(bar_no_add_remaining["name"].iloc[i], "no ads")
                driver.find_element(By.CLASS_NAME, 'hfpxzc').click()
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                case_reviews = soup.find('div', class_ = 'F7nice')
                case_rating = case_reviews.text.split("(")[0]
                case_reviews = case_reviews.text.split("(")[1][:-1]
                #print("try list hit")
                #print(i, bar_no_add_remaining["name"].iloc[i], case_rating)
                #print(i, bar_no_add_remaining["name"].iloc[i], case_reviews)
                pass
        except Exception:
            try:
                # If neither direct hit nor list, try again (moves to exception and sets nan)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                case_reviews = soup.find('div', class_ = 'F7nice')
                case_rating = case_reviews.text.split("(")[0]
                case_reviews = case_reviews.text.split("(")[1][:-1]
                #print("try no hit")
                #print(i, bar_no_add_remaining["name"].iloc[i], case_rating)
                #print(i, bar_no_add_remaining["name"].iloc[i], case_reviews)
            except Exception:
                case_rating = np.nan
                case_reviews = np.nan
                #print("try no hit no rev/rating")
                #print(i, bar_no_add_remaining["name"].iloc[i], case_rating)
                #print(i, bar_no_add_remaining["name"].iloc[i], case_reviews)
                pass
        pass
    try:
        if str("Temporarily closed" in soup.find('div', class_ = 'skqShb').text):
            case_closed = str("Temporarily closed" in soup.find('div', class_ = 'skqShb').text)
        else:
            case_closed = str("Permanently closed" in soup.find('div', class_ = 'skqShb').text)
        #print(i, bar_no_add_remaining["name"].iloc[i], case_closed)
    except Exception:
        case_closed = np.nan
        #print(i, bar_no_add_remaining["name"].iloc[i], "Closed no")
        pass

    try:
        case_price = soup.find('span', class_ = 'mgr77e').text[1:]
        #print(i, bar_no_add_remaining["name"].iloc[i], case_price)
    except Exception:
        case_price = np.nan
        #print(i, bar_no_add_remaining["name"].iloc[i], "Euro no")
        pass
   
        

    poi_url = driver.current_url
    try:
        lat = poi_url.split("@")[1].split(",")[0]
        long = poi_url.split("@")[1].split(",")[1]
    except Exception:
        lat = np.nan
        long = np.nan
        pass
    #print(poi_url)
    row = {"id": bar_id, "name": bar_name, "rating": case_rating, "reviews": case_reviews, "price": case_price, "closed": case_closed, "url": poi_url, "lat": lat, "long": long}
    #print(row)
    bar_reviews_no_add = pd.concat([bar_reviews_no_add, pd.DataFrame(row, index=[i])], ignore_index=True)

    driver.close()
    return bar_reviews_no_add

# Insert number of iterations in range
threadlist = []
with ThreadPoolExecutor(max_workers=4) as executor:
    for i in range(5):
        threadlist.append(executor.submit(get_rating, i))
        time.sleep(1)

wait(threadlist)

bar_reviews_no_add = bar_reviews_no_add.drop_duplicates(subset="id", keep=False)
bar_reviews_no_add.to_csv('data/london/bar_reviews_no_add.csv')

In [ ]:
bar_reviews_no_add.tail(5)

,id,name,rating,reviews,price,closed,url,lat,long
1589,929290848,Heritage Kitchen & Bar,4.6,343,NaN,False,https://www.google.com/maps/place/Heritage+Kit...,51.469364,-0.400025
1590,2652399072,The Fox & Firkin,4.4,"1,149",€€,False,https://www.google.com/maps/place/Fox+%26+Firk...,51.456891,-0.016852
1591,2632761624,Coach & Horses,5.0,6,NaN,False,https://www.google.com/maps/place/Daniela+K%C3...,49.968514,-2.050646
1592,279526311,The Windmill,4.4,"1,688",€€,False,https://www.google.com/maps/place/Windmill/@51...,51.512381,-9.370714
1593,9014316323,Beerblefish - Brewery and Taproom,4.7,74,€€,False,https://www.google.com/maps/place/The+Beerblef...,51.593327,-0.044431
1594,276146932,Crown & Sceptre,4.2,894,€€,False,https://www.google.com/maps/place/The+Crown+%2...,51.762745,-3.895129
1595,982884990,The Adam & Eve,4.5,271,NaN,False,https://www.google.com/maps/place/adam+%26+eve...,53.577155,9.803691
1596,984433443,The Greyhound,5.0,3,NaN,False,https://www.google.com/maps/place/The+Greyhoun...,52.202232,-1.472639
1597,993673927,Ship Inn,4.4,321,NaN,False,https://www.google.com/maps/place/The+Ship+Inn...,52.917498,-5.211004
1598,993781413,Slug & Lettuce,4.4,174,€€,False,https://www.google.com/maps/place/S.L.U.T.+Clu...,52.554697,-1.046957


In [ ]:
bar_reviews_no_add.shape

(1599, 9)

In [ ]:
bar_no_address.shape

(1600, 16)

#### Calculate distance to OSM coordinates

In [ ]:
distance_bars = bar_reviews_no_add.merge(osm_short, on="id", how="inner")
distance_bars.shape

(1599, 19)

In [ ]:
distance_bars = distance_bars.dropna(subset="lat")
distance_bars = distance_bars.dropna(subset="latitude")

In [ ]:
import geopy.distance

In [ ]:
def distance(row):
    return geopy.distance.distance((row["latitude"], row["longitude"]), (row["lat"], row["long"])).km

In [ ]:
distance_bars["distance"] = distance_bars.apply(lambda row: distance(row), axis=1)

In [ ]:
distance_bars.head(5)

In [ ]:
distance_bars[distance_bars["distance"] < 0.3].shape

(537, 20)

## Restaurant review data scraping

In [ ]:
# New DF for restaurants with addresses and names to fill with reviews
restaurant_address = osm_short[(osm_short["amenity"] == "restaurant") & (~osm_short["address"].isna()) & (~osm_short["name"].isna())]
restaurant_address = restaurant_address.reset_index().drop(columns="index")

In [ ]:
# Uncomment both only the first time the code is run.
# restaurant_reviews = bar_reviews[0:0]
# restaurant_reviews.to_csv(('data/london/restaurant_reviews.csv'))

In [ ]:
# "restaurant_address" must be csv on computer
# restaurants, address not nan, web scraping loop with threading

pd.options.mode.chained_assignment = None

restaurant_reviews = pd.read_csv(
    "data/london/restaurant_reviews.csv", index_col="Unnamed: 0")
restaurant_address_remaining = restaurant_address[~restaurant_address["id"].isin(
    restaurant_reviews["id"].to_list())]

start = restaurant_reviews.shape[0]

service = Service(ChromeDriverManager().install())


def driver_setup():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=service, options=options)
    return driver


def get_rating(i):
    global restaurant_reviews
    bar_id = restaurant_address_remaining["id"].iloc[i]
    bar_name = restaurant_address_remaining["name"].iloc[i]
    driver = driver_setup()
    url = f'http://maps.google.com/?q={restaurant_address_remaining["name"].iloc[i]}+{restaurant_address_remaining["address"].iloc[i]}'
    driver.get(url)
    time.sleep(1)

    # Press accept all on cookie question
    try:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()
        time.sleep(1)
    except NoSuchElementException:
        pass

    # Extract raiting/num of reviews, price level and if permanently closed
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        case_reviews = soup.find('div', class_='F7nice')
        case_rating = case_reviews.text.split("(")[0]
        case_reviews = case_reviews.text.split("(")[1][:-1]
        #print(i, restaurant_address_remaining["name"].iloc[i], case_rating)
        #print(i, restaurant_address_remaining["name"].iloc[i], case_reviews)
    except Exception:
        case_rating = np.nan
        case_reviews = np.nan
        #print(i, restaurant_address_remaining["name"].iloc[i], "Rating no")
        pass

    try:
        case_closed = str("Temporarily closed" in soup.find(
            'div', class_='skqShb').text)
        #print(i, restaurant_address_remaining["name"].iloc[i], case_closed)
    except Exception:
        case_closed = np.nan
        #print(i, restaurant_address_remaining["name"].iloc[i], "Closed no")
        pass

    try:
        case_price = soup.find('span', class_='mgr77e').text[1:]
        #print(i, restaurant_address_remaining["name"].iloc[i], case_price)
    except Exception:
        case_price = np.nan
        #print(i, restaurant_address_remaining["name"].iloc[i], "Euro no")
        pass

    poi_url = url
    row = {"id": bar_id, "name": bar_name, "rating": case_rating,
           "reviews": case_reviews, "price": case_price, "closed": case_closed, "url": poi_url}
    #print(row)
    restaurant_reviews = pd.concat(
        [restaurant_reviews, pd.DataFrame(row, index=[i])], ignore_index=True)

    driver.close()
    return restaurant_reviews


# Insert number of iterations wanted in range
threadlist = []
with ThreadPoolExecutor(max_workers=4) as executor:
    for i in range(2):
        threadlist.append(executor.submit(get_rating, i))
        time.sleep(1)

wait(threadlist)

restaurant_reviews = restaurant_reviews.drop_duplicates(
    subset="id", keep=False)
restaurant_reviews.to_csv('data/london/restaurant_reviews.csv')

pd.options.mode.chained_assignment = "warn"

### Upload restaurant with address ratings

In [ ]:
# To SQL

In [ ]:
engine = get_engine()
schema = 'hh_analytics_23_1' # your course schema name, example 'hh_analytics_22_1
table_name = 'g1_restaurant_reviews'

In [ ]:
# Write records stored in poi_gastro to SQL database
if engine!=None:
    try:
        restaurant_reviews.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

The g1_restaurant_reviews table was imported successfully.


In [ ]:
# Give access to all cohort members
group = "student_role_hh_analytics_23_1"
engine = get_engine()
SQL = f"GRANT ALL  ON {schema}.{table_name} to {group}"
with engine.begin() as conn:
    conn.execute(SQL)


## Restaurant without address scraping

In [ ]:
restaurant_reviews = pd.read_csv(
    "data/london/restaurant_reviews.csv", index_col="Unnamed: 0")

In [ ]:
restaurant_no_address = osm_short[(osm_short["amenity"] == "restaurant") & (osm_short["address"].isna()) & (~osm_short["name"].isna())]

In [ ]:
# No working address
restaurant_remaining = restaurant_reviews[(restaurant_reviews["closed"].isna())]

In [ ]:
restaurant_no_address = pd.concat([restaurant_no_address, restaurant_remaining])

In [ ]:
restaurant_no_address = restaurant_no_address.reset_index().drop(columns="index")

In [ ]:
# Uncomment both only the first time the code is run.
# restaurant_reviews_no_add = bar_reviews[0:0]
# restaurant_reviews_no_add.to_csv("data/london/restaurant_reviews_no_add.csv")

In [ ]:
# Choose from list, "bar_reviews_no_address" must be csv on computer
# bars, address not nan, web scraping loop with threading
pd.options.mode.chained_assignment = "warn"

restaurant_reviews_no_add = pd.read_csv("data/london/restaurant_reviews_no_add.csv", index_col="Unnamed: 0")
restaurant_no_add_remaining = restaurant_no_address[~restaurant_no_address["id"].isin(restaurant_reviews_no_add["id"].to_list())]

start = restaurant_reviews_no_add.shape[0]

service = Service(ChromeDriverManager().install())

def driver_setup():
    options = Options()
    #options.add_argument("--headless=new")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--loadExtension="adBlock')
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def get_rating(i):
    global restaurant_reviews_no_add
    restaurant_id = restaurant_no_add_remaining["id"].iloc[i]
    restaurant_name = restaurant_no_add_remaining["name"].iloc[i]
    driver=driver_setup()
    url = f'http://maps.google.com/?ll={restaurant_no_add_remaining["latitude"].iloc[i]},{restaurant_no_add_remaining["longitude"].iloc[i]}'
    driver.get(url)
    time.sleep(1)


    # Press accept all on cookie question
    try:
        driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()
        time.sleep(1)
    except NoSuchElementException:
        pass

    driver.get(url)

    driver.find_element(By.XPATH,'/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]').send_keys(restaurant_no_add_remaining["name"].iloc[i])
    driver.find_element(By.XPATH,'/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]').send_keys(Keys.RETURN)
    time.sleep(4)



    # Extract raiting/num of reviews, price level and if permanently closed
    try:
        # Case for immediate result without list appearing
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        case_reviews = soup.find('div', class_ = 'F7nice')
        case_rating = case_reviews.text.split("(")[0]
        case_reviews = case_reviews.text.split("(")[1][:-1]
        print(i, restaurant_no_add_remaining["name"].iloc[i], case_rating)
        print(i, restaurant_no_add_remaining["name"].iloc[i], case_reviews)
    except Exception:
        try:
            try:
                # Count the number of ads and skip them
                ads = soup.find_all('span', {'class': 'jHLihd'})
                print(ads)
                print(restaurant_no_add_remaining["name"].iloc[i], "len ads", len(ads))
                print(restaurant_no_add_remaining["name"].iloc[i], "ads 1", ads[0])
                if len(ads) > 0:
                    try:
                        print(restaurant_no_add_remaining["name"].iloc[i], "len ads over 0")
                        click_list = driver.find_elements(By.CLASS_NAME, 'hfpxzc')
                        print(restaurant_no_add_remaining["name"].iloc[i], click_list[len(ads)])
                        click_list[len(ads)].click()
                        time.sleep(2)
                        soup = BeautifulSoup(driver.page_source, 'html.parser')
                        case_reviews = soup.find('div', class_ = 'F7nice')
                        case_rating = case_reviews.text.split("(")[0]
                        case_reviews = case_reviews.text.split("(")[1][:-1]
                    except Exception:
                        pass
            except Exception:
                # Case if no ads
                print(restaurant_no_add_remaining["name"].iloc[i], "no ads")
                driver.find_element(By.CLASS_NAME, 'hfpxzc').click()
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                case_reviews = soup.find('div', class_ = 'F7nice')
                case_rating = case_reviews.text.split("(")[0]
                case_reviews = case_reviews.text.split("(")[1][:-1]
                print("try list hit")
                print(i, restaurant_no_add_remaining["name"].iloc[i], case_rating)
                print(i, restaurant_no_add_remaining["name"].iloc[i], case_reviews)
        except Exception:
            try:
                # If neither direct hit nor list, try again (moves to exception and sets nan)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                case_reviews = soup.find('div', class_ = 'F7nice')
                case_rating = case_reviews.text.split("(")[0]
                case_reviews = case_reviews.text.split("(")[1][:-1]
                print("try no hit")
                print(i, restaurant_no_add_remaining["name"].iloc[i], case_rating)
                print(i, restaurant_no_add_remaining["name"].iloc[i], case_reviews)
            except Exception:
                case_rating = np.nan
                case_reviews = np.nan
                print("try no hit no rev/rating")
                print(i, restaurant_no_add_remaining["name"].iloc[i], case_rating)
                print(i, restaurant_no_add_remaining["name"].iloc[i], case_reviews)
    try:
        if str("Temporarily closed" in soup.find('div', class_ = 'skqShb').text):
            case_closed = str("Temporarily closed" in soup.find('div', class_ = 'skqShb').text)
        else:
            case_closed = str("Permanently closed" in soup.find('div', class_ = 'skqShb').text)
        print(i, restaurant_no_add_remaining["name"].iloc[i], case_closed)
    except Exception:
        case_closed = np.nan
        print(i, restaurant_no_add_remaining["name"].iloc[i], "Closed no")
        pass

    try:
        case_price = soup.find('span', class_ = 'mgr77e').text[1:]
        print(i, restaurant_no_add_remaining["name"].iloc[i], case_price)
    except Exception:
        case_price = np.nan
        print(i, restaurant_no_add_remaining["name"].iloc[i], "Euro no")
        pass
   
        

    poi_url = driver.current_url
    try:
        lat = poi_url.split("@")[1].split(",")[0]
        long = poi_url.split("@")[1].split(",")[1]
    except Exception:
        lat = np.nan
        long = np.nan
    print(poi_url)
    row = {"id": restaurant_id, "name": restaurant_name, "rating": case_rating, "reviews": case_reviews, "price": case_price, "closed": case_closed, "url": poi_url, "lat": lat, "long": long}
    print(row)
    restaurant_reviews_no_add = pd.concat([restaurant_reviews_no_add, pd.DataFrame(row, index=[i])], ignore_index=True)

    #driver.close()
    return restaurant_reviews_no_add

# Insert number of iterations in range
threadlist = []
with ThreadPoolExecutor(max_workers=4) as executor:
    for i in range(4):
        threadlist.append(executor.submit(get_rating, i))
        time.sleep(1)

wait(threadlist)

restaurant_reviews_no_add = restaurant_reviews_no_add.drop_duplicates(subset="id", keep=False)
restaurant_reviews_no_add.to_csv('data/london/restaurant_reviews_no_add.csv')

[]
Central Restaurant len ads 0
Central Restaurant no ads
[]
Karma Lounge len ads 0
Karma Lounge no ads
2 Old Tree Daiwan Bee 3.4
2 Old Tree Daiwan Bee 553
2 Old Tree Daiwan Bee False
2 Old Tree Daiwan Bee €
https://www.google.com/maps/place/Old+Tree+Daiwan+Bee/@51.5110545,-0.1332281,19z/data=!4m6!3m5!1s0x487604d3b7ddd157:0xa7ccf5bb73d1992f!8m2!3d51.5110545!4d-0.1325844!16s%2Fg%2F1yh805m8l
{'id': 26845558, 'name': 'Old Tree Daiwan Bee', 'rating': '3.4', 'reviews': '553', 'price': '€', 'closed': 'False', 'url': 'https://www.google.com/maps/place/Old+Tree+Daiwan+Bee/@51.5110545,-0.1332281,19z/data=!4m6!3m5!1s0x487604d3b7ddd157:0xa7ccf5bb73d1992f!8m2!3d51.5110545!4d-0.1325844!16s%2Fg%2F1yh805m8l', 'lat': '51.5110545', 'long': '-0.1332281'}
try list hit
0 Central Restaurant 4.7
0 Central Restaurant 321
0 Central Restaurant False
0 Central Restaurant €€
https://www.google.com/maps/place/The+House+Restaurant/@51.6020306,-0.1935029,19z/data=!4m10!1m2!2m1!1sCentral+Restaurant!3m6!1s0x487617544

## Cafe reviews

In [ ]:
# New DF for cafes with addresses and names to fill with reviews
cafe_address = osm_short[(osm_short["amenity"] == "cafe") & (~osm_short["address"].isna()) & (~osm_short["name"].isna())]
cafe_address = cafe_address.reset_index().drop(columns="index")

In [85]:
# Uncomment both only the first time the code is run
# cafe_reviews = bar_reviews[0:0]
# cafe_reviews.to_csv(('data/london/cafe_reviews.csv'))

In [ ]:
# "cafe_address" must be csv on computer
# restaurants, address not nan, web scraping loop with threading

pd.options.mode.chained_assignment = None

cafe_reviews = pd.read_csv(
    "data/london/cafe_reviews.csv", index_col="Unnamed: 0")
cafe_address_remaining = cafe_address[~cafe_address["id"].isin(
    cafe_reviews["id"].to_list())]

start = cafe_reviews.shape[0]

service = Service(ChromeDriverManager().install())


def driver_setup():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=service, options=options)
    return driver


def get_rating(i):
    global cafe_reviews
    cafe_id = cafe_address_remaining["id"].iloc[i]
    cafe_name = cafe_address_remaining["name"].iloc[i]
    driver = driver_setup()
    url = f'http://maps.google.com/?q={cafe_address_remaining["name"].iloc[i]}+{cafe_address_remaining["address"].iloc[i]}'
    driver.get(url)
    time.sleep(1)

    # Press accept all on cookie question
    try:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()
        time.sleep(1)
    except NoSuchElementException:
        pass

    # Extract raiting/num of reviews, price level and if permanently closed
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        case_reviews = soup.find('div', class_='F7nice')
        case_rating = case_reviews.text.split("(")[0]
        case_reviews = case_reviews.text.split("(")[1][:-1]
        #print(i, cafe_address_remaining["name"].iloc[i], case_rating)
        #print(i, cafe_address_remaining["name"].iloc[i], case_reviews)
    except Exception:
        case_rating = np.nan
        case_reviews = np.nan
        #print(i, cafe_address_remaining["name"].iloc[i], "Rating no")
        pass

    try:
        case_closed = str("Temporarily closed" in soup.find(
            'div', class_='skqShb').text)
        #print(i, cafe_address_remaining["name"].iloc[i], case_closed)
    except Exception:
        case_closed = np.nan
        #print(i, cafe_address_remaining["name"].iloc[i], "Closed no")
        pass

    try:
        case_price = soup.find('span', class_='mgr77e').text[1:]
        #print(i, cafe_address_remaining["name"].iloc[i], case_price)
    except Exception:
        case_price = np.nan
        #print(i, cafe_address_remaining["name"].iloc[i], "Euro no")
        pass

    poi_url = url
    row = {"id": cafe_id, "name": cafe_name, "rating": case_rating,
           "reviews": case_reviews, "price": case_price, "closed": case_closed, "url": poi_url}
    #print(row)
    cafe_reviews = pd.concat(
        [cafe_reviews, pd.DataFrame(row, index=[i])], ignore_index=True)

    driver.close()
    return cafe_reviews


# Insert number of iterations wanted in range
threadlist = []
with ThreadPoolExecutor(max_workers=4) as executor:
    for i in range(50):
        threadlist.append(executor.submit(get_rating, i))
        time.sleep(1)

wait(threadlist)

cafe_reviews = cafe_reviews.drop_duplicates(
    subset="id", keep=False)
cafe_reviews.to_csv('data/london/cafe_reviews.csv')

pd.options.mode.chained_assignment = "warn"

## fast_food reviews

In [ ]:
# New DF for cafes with addresses and names to fill with reviews
fast_food_address = osm_short[(osm_short["amenity"] == "fast_food") & (~osm_short["address"].isna()) & (~osm_short["name"].isna())]
fast_food_address = fast_food_address.reset_index().drop(columns="index")

In [86]:
# Uncomment both only the first time the code is run.
# fast_food_reviews = bar_reviews[0:0]
# fast_food_reviews.to_csv(('data/london/fast_food_reviews.csv'))

In [ ]:
# "fast_food_address" must be csv on computer
# restaurants, address not nan, web scraping loop with threading

pd.options.mode.chained_assignment = None

fast_food_reviews = pd.read_csv(
    "data/london/fast_food_reviews.csv", index_col="Unnamed: 0")
fast_food_address_remaining = fast_food_address[~fast_food_address["id"].isin(
    fast_food_reviews["id"].to_list())]

start = fast_food_reviews.shape[0]

service = Service(ChromeDriverManager().install())


def driver_setup():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=service, options=options)
    return driver


def get_rating(i):
    global fast_food_reviews
    fast_food_id = fast_food_address_remaining["id"].iloc[i]
    fast_food_name = fast_food_address_remaining["name"].iloc[i]
    driver = driver_setup()
    url = f'http://maps.google.com/?q={fast_food_address_remaining["name"].iloc[i]}+{fast_food_address_remaining["address"].iloc[i]}'
    driver.get(url)
    time.sleep(1)

    # Press accept all on cookie question
    try:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()
        time.sleep(1)
    except NoSuchElementException:
        pass

    # Extract raiting/num of reviews, price level and if permanently closed
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        case_reviews = soup.find('div', class_='F7nice')
        case_rating = case_reviews.text.split("(")[0]
        case_reviews = case_reviews.text.split("(")[1][:-1]
        #print(i, fast_food_address_remaining["name"].iloc[i], case_rating)
        #print(i, fast_food_address_remaining["name"].iloc[i], case_reviews)
    except Exception:
        case_rating = np.nan
        case_reviews = np.nan
        #print(i, fast_food_address_remaining["name"].iloc[i], "Rating no")
        pass

    try:
        case_closed = str("Temporarily closed" in soup.find(
            'div', class_='skqShb').text)
        #print(i, fast_food_address_remaining["name"].iloc[i], case_closed)
    except Exception:
        case_closed = np.nan
        #print(i, fast_food_address_remaining["name"].iloc[i], "Closed no")
        pass

    try:
        case_price = soup.find('span', class_='mgr77e').text[1:]
        #print(i, fast_food_address_remaining["name"].iloc[i], case_price)
    except Exception:
        case_price = np.nan
        #print(i, fast_food_address_remaining["name"].iloc[i], "Euro no")
        pass

    poi_url = url
    row = {"id": fast_food_id, "name": fast_food_name, "rating": case_rating,
           "reviews": case_reviews, "price": case_price, "closed": case_closed, "url": poi_url}
    #print(row)
    fast_food_reviews = pd.concat(
        [fast_food_reviews, pd.DataFrame(row, index=[i])], ignore_index=True)

    driver.close()
    return fast_food_reviews


# Insert number of iterations wanted in range
threadlist = []
with ThreadPoolExecutor(max_workers=4) as executor:
    for i in range(10):
        threadlist.append(executor.submit(get_rating, i))
        time.sleep(1)

wait(threadlist)

fast_food_reviews = fast_food_reviews.drop_duplicates(
    subset="id", keep=False)
fast_food_reviews.to_csv('data/london/fast_food_reviews.csv')

pd.options.mode.chained_assignment = "warn"